In [ ]:
bx_cat = (df['bx'] > np.percentile(df['bx'], 95)).astype(int)

from GG_mthesis.d01_data.processing import rmap 
projection = ccrs.PlateCarree()
fig = plt.figure(figsize=(16,8))
ax = plt.axes(projection=projection)
bx = ax.scatter(df['lon'], df['lat'], c=bx_cat, s=0.1,transform=ccrs.PlateCarree(), zorder=500, cmap='binary',
                #vmax=np.percentile(df['bx'], 95), vmin=np.percentile(df['bx'], 5)
                )
ax.coastlines(zorder=100)
ax.add_feature(cartopy.feature.LAND, color='green', zorder=50)
gl = ax.gridlines(draw_labels=True, linestyle='-.', color='gray')
gl.xlabels_top = False
gl.ylabels_right = False
fig.colorbar(bx)

fronts=rmap.load_fronts('data/01_raw/ACC_FRONTS/PARK/62985.nc')
fronts=rmap.sel_front_region(fronts, lon_min=-70, lon_max=-50, lat_min=-65, lat_max=-55)
s=4
#SAF = ax.plot(fronts.loc['SAF']['longitude'], fronts.loc['SAF']['latitude'],transform=ccrs.PlateCarree(), label='SAF_Park', color='purple')#, s=s)
PF = ax.plot(fronts.loc['PF']['longitude'], fronts.loc['PF']['latitude'], transform=ccrs.PlateCarree(), label='PF_Park', color='black')#, s=s)
SACCF = ax.plot(fronts.loc['SACCF']['longitude'], fronts.loc['SACCF']['latitude'],transform=ccrs.PlateCarree(), label='SACCF_Park', color='yellow')#, s=s)
NB = ax.plot(fronts.loc['NB']['longitude'], fronts.loc['NB']['latitude'],transform=ccrs.PlateCarree(), label='NB_Park', color='red')#, s=s)
SB = ax.plot(fronts.loc['SB']['longitude'], fronts.loc['SB']['latitude'],transform=ccrs.PlateCarree(), label='SB_Park', color='red')#, s=s)
#ax.set_extent([-63.3, -63.5, -58.5, -58.4])
ax.set_extent([-66,-60,-61.5,-57])

In [ ]:
df_bx_rolling = df_90.rolling(20).mean()
df_90 = df[df['bx'] >= np.percentile(df['bx'], 98)]

fig, (ax0, ax1) = plt.subplots(figsize=(16,12), nrows=2, sharex=True)
ax0.scatter(df_90['mdt'], df_90['bx'])
ax1.scatter(df_bx_rolling['mdt'], df_bx_rolling['bx'])
for ax in [ax0, ax1]:
    ax.vlines(-0.1, 0, 1E-6,
            color='black',
            linestyle='-.')
    
    ax.vlines(-0.58, 0, 1E-6,
        color='black',
        linestyle='-.')

    ax.vlines(-1.0, 0, 1E-6,
    color='black',
    linestyle='-.')

In [ ]:
from scipy import signal

sig = signal.detrend(df.sort_values('adt_mean')['bx'])

fig, ax = plt.subplots(figsize=(16,8))
ax.semilogy(df.sort_values('adt_mean')['adt_mean'], sig)

In [ ]:
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

In [ ]:
IQR = np.percentile(dataset['bx'], 75) - np.percentile(dataset['bx'], 25)
upper_whisker = np.percentile(dataset['bx'], 75)+1.5*IQR
lower_whisker = np.percentile(dataset['bx'], 25)-1.5*IQR

dataset_high_pos = dataset[dataset['bx'] >= upper_whisker]
dataset_high_pos['cat'] = 'high'
dataset_high_neg = dataset[dataset['bx'] <= lower_whisker]
dataset_high_neg['cat'] = 'high'
dataset_high = pd.concat([dataset_high_pos,dataset_high_neg]).sort_values('adt_mean')

dataset_mid_pos = dataset[(dataset['bx'] < upper_whisker) & (dataset['bx'] >= np.percentile(dataset['bx'], 75))]
dataset_mid_pos['cat'] = 'mid'
dataset_mid_neg = dataset[(dataset['bx'] > lower_whisker) & (dataset['bx'] <= np.percentile(dataset['bx'], 25))]
dataset_mid_pos['cat'] = 'mid'
dataset_mid = pd.concat([dataset_mid_pos,dataset_mid_neg]).sort_values('adt_mean')

dataset_low = dataset[(dataset['bx'] < np.percentile(dataset['bx'], 75)) & (dataset['bx'] > np.percentile(dataset['bx'], 25))]
dataset_low['cat'] = 'low'
dataset_low = dataset_low.sort_values('adt_mean')

dataset = pd.concat([dataset_high, dataset_mid, dataset_low]).sort_values('adt_mean')


fig, (ax0, ax1, ax2) = plt.subplots(nrows= 3, figsize=(16,8))
ax0.plot(dataset_high['adt_mean'], dataset_high['bx'])
ax1.plot(dataset_mid['adt_mean'], dataset_mid['bx'])
ax2.plot(dataset_low['adt_mean'], dataset_low['bx'])

In [ ]:
def scatter_hist(x, y, ax, ax_histx, ax_histy):
    # no labels
    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)

    # the scatter plot:
    ax.scatter(x, y, c='black')

    # now determine nice limits by hand:
    binwidth = 0.1
    xymax = max(np.max(np.abs(x)), np.max(np.abs(y)))
    lim = (int(xymax/binwidth) + 1) * binwidth

    bins = np.arange(-lim, lim + binwidth, binwidth)
    ax_histx.hist(x, bins=bins, color='black')
    ax_histy.hist(y, bins=bins, orientation='horizontal', color='black')

x = dataframe.mdt
y = dataframe.log_bx_abs

# Start with a square Figure.
fig = plt.figure(figsize=(12, 12))
# Add a gridspec with two rows and two columns and a ratio of 1 to 4 between
# the size of the marginal axes and the main axes in both directions.
# Also adjust the subplot parameters for a square plot.
gs = fig.add_gridspec(2, 2,  width_ratios=(4, 1), height_ratios=(1, 4),
                      left=0.1, right=0.9, bottom=0.1, top=0.9,
                      wspace=0.05, hspace=0.05)
# Create the Axes.
ax = fig.add_subplot(gs[1, 0])
ax_histx = fig.add_subplot(gs[0, 0], sharex=ax)
ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)
# Draw the scatter plot and marginals.
scatter_hist(x, y, ax, ax_histx, ax_histy)
ax.set_ylim(y.min(),y.max())
ax.set_xlim(x.min(), x.max())

In [ ]:
dataset = dataframe[dataframe['bx_abs'] < 2E-6]
ol1, wh1, iq1 = filter(dataframe)
ol1_ol2, ol1_wh2, ol1_iq2 = filter(ol1)
wh1_ol2, wh1_wh2, wh1_iq2 = filter(wh1)
iq1_ol2, iq1_wh2, iq1_iq2 = filter(iq1)

In [ ]:
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[1,0])
ax3 = fig.add_subplot(gs[1,1])
ax4 = fig.add_subplot(gs[2,0])
ax5 = fig.add_subplot(gs[2,1])

In [ ]:
def scatter_hist(x, y, ax, ax_histx, ax_histy):
    # no labels
    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)

    # the scatter plot:
    ax.scatter(x, y, c=wh1.lat)

    # now determine nice limits by hand:
    binwidth = 0.1
    xymax = max(np.max(np.abs(x)), np.max(np.abs(y)))
    lim = (int(xymax/binwidth) + 1) * binwidth

    bins = np.arange(-lim, lim + binwidth, binwidth)
    ax_histx.hist(x, bins=bins, color='black')
    ax_histy.hist(y, bins=bins, orientation='horizontal', color='black')

x = wh1_1.mdt
y = wh1.log_bx_abs

# Start with a square Figure.
fig = plt.figure(figsize=(12, 12))
# Add a gridspec with two rows and two columns and a ratio of 1 to 4 between
# the size of the marginal axes and the main axes in both directions.
# Also adjust the subplot parameters for a square plot.
gs = fig.add_gridspec(2, 2,  width_ratios=(4, 1), height_ratios=(1, 4),
                      left=0.1, right=0.9, bottom=0.1, top=0.9,
                      wspace=0.05, hspace=0.05)
# Create the Axes.
ax = fig.add_subplot(gs[1, 0])
ax_histx = fig.add_subplot(gs[0, 0], sharex=ax)
ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)
# Draw the scatter plot and marginals.
scatter_hist(x, y, ax, ax_histx, ax_histy)
ax.set_ylim(y.min(),y.max())
ax.set_xlim(x.min(), x.max())

In [ ]:
#df_sel = dataset[dataset['lon'] > -65.0][dataset['lon'] < -62]

ds_mdt_sel = ds_mdt.sel(longitude=slice(-65,-62), latitude=slice(-62,-55))
ds_adt_mean_sel = ds_adt_mean.sel(longitude=slice(-65,-62), latitude=slice(-62,-55))
#df_sel = df2[df2['lon'] > -65.5][df2['lon'] < -62]

#dataframe = df_sel[df_sel['bx_abs'] < np.percentile(df_sel['bx'],99)]
#dataframe = df_sel
dataframe = ol
lat_min = dataframe['lat'].min()+0.1
lat_max = dataframe['lat'].max()-0.1
lon_min = dataframe['lon'].min()+0.1
lon_max = dataframe['lon'].max()-0.1
lon_grid = np.arange(lon_min, lon_max, 0.1)
lat_grid = np.arange(lat_min, lat_max, 0.1)
x_grid, y_grid = np.meshgrid(lon_grid,lat_grid)
xi = (x_grid, y_grid)

x = dataframe['lon'].to_numpy()
y = dataframe['lat'].to_numpy()
points =  dataframe[['lon','lat']].to_numpy() #np.array([df2['lon'].to_numpy(), df2['lat'].to_numpy()])
values = (dataframe['bx_abs']).to_numpy()

from scipy import interpolate
interpol = interpolate.griddata((x, y), values, xi, method='linear')

from matplotlib import ticker
fig, ax = plt.subplots(figsize=(16,8))
contf = ax.contourf(x_grid, y_grid, interpol, levels=30, cmap='coolwarm')
#, normalize='linear')
fig.colorbar(contf)
#ax.scatter(wh['lon'], wh['lat'])
#ax.contour(ds_mdt_sel['longitude'], ds_mdt_sel['latitude'], ds_mdt_sel['mdt'], colors='black', levels= 10)
ax.contour(ds_adt_mean_sel['longitude'], ds_adt_mean_sel['latitude'], ds_adt_mean_sel['adt'], colors='black', levels= 10)

In [ ]:
# libraries
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import kde

top = ol
bottom = iq

# Outliers
x0 = top['mdt']
y0 = top['log_bx_abs']

nbins=100
k0 = kde.gaussian_kde([x0,y0])
xi0, yi0 = np.mgrid[x0.min():x0.max():nbins*1j, y0.min():y0.max():nbins*1j]
zi0 = k0(np.vstack([xi0.flatten(), yi0.flatten()]))

# Whiskers
x1 = bottom['mdt']
y1 = bottom['log_bx_abs']

nbins=100
k1 = kde.gaussian_kde([x1,y1])
xi1, yi1 = np.mgrid[x1.min():x1.max():nbins*1j, y1.min():y1.max():nbins*1j]
zi1 = k(np.vstack([xi1.flatten(), yi1.flatten()]))
 
# Make the plot
fig, (ax0, ax1) = plt.subplots(nrows=2, figsize=(16,8), sharex=True)
pc0 = ax0.pcolormesh(xi0, yi0, zi0.reshape(xi.shape), shading='auto', cmap='coolwarm')
fig.colorbar(pc0, location='right', pad=0.01, ax=ax0)
ax0.scatter(top['mdt'], top['log_bx_abs'], c='black')

pc1 = ax1.pcolormesh(xi1, yi1, zi1.reshape(xi.shape), shading='auto', cmap='coolwarm')
fig.colorbar(pc1, location='right', pad=0.01, ax=ax1)
ax1.scatter(bottom['mdt'], bottom['log_bx_abs'], c='black')

In [ ]:
data_aux = pd.DataFrame()
cruise_ids = df_sel['cruise_id'].unique()
for id in cruise_ids:
    df_cruise = df_sel[df_sel['cruise_id'] == id]
    print(id)
    temp_aux = df_cruise['CT'].diff()
    sal_aux = df_cruise['CT'].diff()
    aux = pd.DataFrame({'temp_diff': temp_aux, 'sal_diff':sal_aux})
    data_aux = data_aux.append(aux)

In [ ]:
T = np.linspace(df_sel['CT'].min(), df_sel['CT'].max(), 100)
S = np.linspace(df_sel['SA'].min(), df_sel['SA'].max(), 100)
T_diff = np.arange(-6, 8, 0.1)
S_diff = np.arange(-6, 8, 0.1)
rho_T = gsw.rho(35, T, 0)
rho_S = gsw.rho(S, 5, 0)
bx_T = (g/rho_0)*(T_diff)/1000
bx_S = (g/rho_0)*(np.diff(rho_S))/1000

fig, (ax0, ax1) = plt.subplots(nrows=2, figsize=(12,6))
ax0.plot(T_diff, bx_T, c='black')
ax1.plot(S, rho_S, c='black')

In [ ]:
T_diff = np.linspace(data_aux['temp_diff'].min()-0.1,data_aux['temp_diff'].max()+0.1,1000)
S_diff = np.linspace(data_aux['sal_diff'].min()-0.1,data_aux['sal_diff'].max()+0.1,1000)

T = np.linspace(df_sel['CT'].min()-0.1,df_sel['CT'].max()+0.1,1000)
S = np.linspace(df_sel['SA'].min()-0.1,df_sel['SA'].max()+0.1,1000)

Tg, Sg = np.meshgrid(T,S)
rho =gsw.rho(Sg,Tg, 0)
bx = (g/rho_0)*(np.diff(rho))/1000

#-----------------------------------------------------------------------------#

fig, ax0 = plt.subplots( figsize=(16,8))

#-----------------------------------------------------------------------------#

#ax0
cs = ax0.contour(S_diff[1:], T_diff, bx, colors='gray', zorder=1, levels= 10)
cl = plt.clabel(cs, fontsize=14, inline=True)

#sc = ax0.contour(S, T, bx)
#fp = ax0.plot(salL, gsw.CT_freezing(salL, 0, 0), color='royalblue')
#cb = plt.colorbar(sc, ax=ax0)

ax0.set(xlabel='Salinity [$g/kg$]', ylabel='Temperature [$°C$]')
#ax0.xaxis.set_major_locator(MaxNLocator(nbins=6))
#ax0.yaxis.set_major_locator(MaxNLocator(nbins=8))
ax0.tick_params(direction='out')
#cb.set_label('Density [kg m$^{-3}$]')
#cb.set_label('Latitude [°N]')

In [ ]:

projection =  ccrs.SouthPolarStereo(central_longitude=-50)
transform = ccrs.PlateCarree() #ccrs.SouthPolarStereo()

fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection=projection)
ax.coastlines(zorder=100)
#ax.add_feature(cartopy.feature.LAND, color='green', zorder=50)

#ax.plot([-64.4,-64.3], [-55.65, -55.55], c='red')
#ax.plot([-64.3, -58.9], [-55.55, -60.5], c='red')
#ax.plot([-58.9, -59.15], [-60.5, -60.6], c='red')
#ax.plot([-59.15, -64.4], [-60.6,-55.65], c='red')

lon_min_lim = 0
lon_max_lim = 180
lat_min_lim = -70
lat_max_lim = -30

ax.scatter([lon_min_lim, lon_min_lim, lon_max_lim, lon_max_lim], [lat_min_lim, lat_max_lim, lat_min_lim, lat_max_lim], s=0, c='r', transform=transform)

ax.gridlines(lw=1, ls='-.', ec='black', draw_labels=True)


In [ ]:
#Map
from GG_mthesis.d01_data.processing import rmap 
projection = ccrs.PlateCarree() #ccrs.SouthPolarStereo()
transform = ccrs.PlateCarree() #ccrs.SouthPolarStereo()

fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection=projection)

#bathy
filepath_bath='data/01_raw/ETOPO1_Bed_g_gmt4.grd'
bath = rmap.load(filepath_bath)
lon_min = -67.5
lon_max = -57.5
lat_min = -63
lat_max = -55
bath = rmap.select_region(bath, lon_min, lon_max, lat_min, lat_max)
import cmocean.cm as cm
x = bath.x
y = bath.y
z = bath.z
vmin = z.min()
vmax = z.max()
topo_f = ax.contourf(x, y, z, 
        cmap = cm.deep_r,
            vmin = vmin,
            vmax = 0,
            transform=transform ,
        #    norm=MidpointNormalize(
        #        midpoint=-1000,
        #        vmin=vmin,lines
        #        vmax=0
        #     ),
        levels=20
        )

bx = ax.scatter(df['lon'], df['lat'], c='black', s=0.1,transform=transform, zorder=50,
                #vmax=np.percentile(df_test['bx'], 95), vmin=np.percentile(df_test['bx'], 5)
                )
ax.coastlines(zorder=100)
ax.add_feature(cartopy.feature.LAND, color='green', zorder=50)
gl = ax.gridlines(draw_labels=True, linestyle='-.', color='gray')
#ax.set_xticks(np.arange(-66.5,-56,0.5), crs=ccrs.PlateCarree())
#plt.xticks(rotation = 90)
#ax.set_xlim([-67.5, -57.5])
#ax.set_ylim([-63, -55])
gl.xlabel_style = {'rotation': 90}
#gl.xlabels_bottom = False
gl.xlabels_top = False
gl.ylabels_right = False
#fig.colorbar(bx)

#polygon_df.boundary.plot(color='red', linewidth=2, zorder=100, ax=ax, figsize=(20,10))

polygon_df_1.boundary.plot(color='red', linewidth=2, zorder=100, ax=ax, figsize=(20,10), transform=transform)
polygon_df_2.boundary.plot(color='yellow', linewidth=2, zorder=200, ax=ax, figsize=(20,10), transform=transform)
polygon_df_3.boundary.plot(color='lightgreen', linewidth=2, zorder=200, ax=ax, figsize=(20,10), transform=transform)
polygon_df_4.boundary.plot(color='purple', linewidth=2, zorder=200, ax=ax, figsize=(20,10), transform=transform)
polygon_df_5.boundary.plot(color='orange', linewidth=2, zorder=200, ax=ax, figsize=(20,10), transform=transform)
polygon_df_6.boundary.plot(color='white', linewidth=2, zorder=200, ax=ax, figsize=(20,10), transform=transform)


#ax.plot([-64.4,-64.3], [-55.65, -55.55], c='red')
#ax.plot([-64.3, -58.9], [-55.55, -60.5], c='red')
#ax.plot([-58.9, -59.15], [-60.5, -60.6], c='red')
#ax.plot([-59.15, -64.4], [-60.6,-55.65], c='red')


#fronts=rmap.load_fronts('data/01_raw/ACC_FRONTS/PARK/62985.nc')
#fronts=rmap.sel_front_region(fronts, lon_min, lon_max, lat_min, lat_max)
#s=4
#SAF = ax.plot(fronts.loc['SAF']['longitude'], fronts.loc['SAF']['latitude'],transform=ccrs.PlateCarree(), label='SAF_Park', color='purple')#, s=s)
#PF = ax.plot(fronts.loc['PF']['longitude'], fronts.loc['PF']['latitude'], transform=ccrs.PlateCarree(), label='PF_Park', color='black')#, s=s)
#SACCF = ax.plot(fronts.loc['SACCF']['longitude'], fronts.loc['SACCF']['latitude'],transform=ccrs.PlateCarree(), label='SACCF_Park', color='yellow')#, s=s)
#NB = ax.plot(fronts.loc['NB']['longitude'], fronts.loc['NB']['latitude'],transform=ccrs.PlateCarree(), label='NB_Park', color='red')#, s=s)
#SB = ax.plot(fronts.loc['SB']['longitude'], fronts.loc['SB']['latitude'],transform=ccrs.PlateCarree(), label='SB_Park', color='red')#, s=s)


#from mpl_toolkits.axes_grid.inset_locator import inset_axes

In [ ]:
#Map
from GG_mthesis.d01_data.processing import rmap 
projection = ccrs.SouthPolarStereo(central_longitude=-65)
transform = ccrs.PlateCarree() #ccrs.SouthPolarStereo()

from matplotlib.gridspec import GridSpec

fig = plt.figure(figsize=(20,10))
ax = plt.axes(projection=projection)

lon_min_lim = -68
lon_max_lim = -58
lat_min_lim = -63
lat_max_lim = -55

ax.scatter([lon_min_lim, lon_min_lim, lon_max_lim, lon_max_lim], [lat_min_lim, lat_max_lim, lat_min_lim, lat_max_lim], s=0, c='r', transform=transform)  
                         

bx1 = ax.scatter(df_sel_1['lon'], df_sel_1['lat'], c='black', s=0.1,transform=transform, zorder=50,
                #vmax=np.percentile(df_test['bx'], 95), vmin=np.percentile(df_test['bx'], 5)
                )
bx2 = ax.scatter(df_sel_2['lon'], df_sel_2['lat'], c='black', s=0.1,transform=transform, zorder=50,
                #vmax=np.percentile(df_test['bx'], 95), vmin=np.percentile(df_test['bx'], 5)
                )
bx3 = ax.scatter(df_sel_3['lon'], df_sel_3['lat'], c='black', s=0.1,transform=transform, zorder=50,
                #vmax=np.percentile(df_test['bx'], 95), vmin=np.percentile(df_test['bx'], 5)
                )
bx4 = ax.scatter(df_sel_4['lon'], df_sel_4['lat'], c='black', s=0.1,transform=transform, zorder=50,
                #vmax=np.percentile(df_test['bx'], 95), vmin=np.percentile(df_test['bx'], 5)
                )
bx5 = ax.scatter(df_sel_5['lon'], df_sel_5['lat'], c='black', s=0.1,transform=transform, zorder=50,
                #vmax=np.percentile(df_test['bx'], 95), vmin=np.percentile(df_test['bx'], 5)
                )
bx6 = ax.scatter(df_sel_6['lon'], df_sel_6['lat'], c='black', s=0.1,transform=transform, zorder=50,
                #vmax=np.percentile(df_test['bx'], 95), vmin=np.percentile(df_test['bx'], 5)
                )
ax.coastlines(zorder=100)
ax.add_feature(cartopy.feature.LAND, color='green', zorder=50)
gl = ax.gridlines(draw_labels=True, x_inline=False, y_inline=False, linestyle='-.', color='gray')
#ax.set_xticks(np.arange(-66.5,-56,0.5), crs=ccrs.PlateCarree())
#plt.xticks(rotation = 90)
gl.xlabel_style = {'rotation': 90}
#gl.xlabels_bottom = False
#gl.xlabels_top = False
#gl.ylabels_right = False
#fig.colorbar(bx)

#polygon_df_1.boundary.plot(color='red', linewidth=1, zorder=100, ax=ax, transform=transform)
#polygon_df_2.boundary.plot(color='yellow', linewidth=1, zorder=200, ax=ax, transform=transform)
#polygon_df_3.boundary.plot(color='lightgreen', linewidth=1, zorder=200, ax=ax, transform=transform)
#polygon_df_4.boundary.plot(color='purple', linewidth=1, zorder=200, ax=ax, transform=transform)
#polygon_df_5.boundary.plot(color='orange', linewidth=1, zorder=200, ax=ax, transform=transform)
#polygon_df_6.boundary.plot(color='red', linewidth=1, zorder=200, ax=ax, transform=transform)
fig.set_figwidth(20)

#bathy
filepath_bath='data/01_raw/ETOPO1_Bed_g_gmt4.grd'
bath = rmap.load(filepath_bath)
lon_min = -70
lon_max = -55
lat_min = -67
lat_max = -53
bath = rmap.select_region(bath, lon_min, lon_max, lat_min, lat_max)
import cmocean.cm as cm
x = bath.x
y = bath.y
z = bath.z
vmin = z.min()
vmax = z.max()
topo_f = ax.contourf(x, y, z, 
        cmap = cm.deep_r,
            vmin = vmin,
            vmax = 0,
            transform=transform ,
        #    norm=MidpointNormalize(
        #        midpoint=-1000,
        #        vmin=vmin,lines
        #        vmax=0
        #     ),
        levels=20
        )

#extent
lon_min = -67
lon_max = -58
lat_min = -63
lat_max = -55
ax.set_extent([lon_min, lon_max, lat_min, lat_max])

#inset
import matplotlib.path as mpath
projection = ccrs.SouthPolarStereo(central_longitude=-65)

ins = ax.inset_axes([0.47,0.65,0.495,0.495], projection=projection)

ins.coastlines(zorder=100)
#ax.add_feature(cartopy.feature.LAND, color='green', zorder=50)

#ax.plot([-64.4,-64.3], [-55.65, -55.55], c='red')
#ax.plot([-64.3, -58.9], [-55.55, -60.5], c='red')
#ax.plot([-58.9, -59.15], [-60.5, -60.6], c='red')
#ax.plot([-59.15, -64.4], [-60.6,-55.65], c='red')

ins.gridlines(lw=1, ls='-.', ec='black', draw_labels=False)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

ins.set_extent([-90, -30, -70, -50], ccrs.PlateCarree())
#ins.set_boundary(circle, transform=ins.transAxes)

ins.plot([-67,-67, -58, -58, -67], [-63, -55, -55, -63, -63], c='red', transform=ccrs.PlateCarree())

In [ ]:
lon_min = -75
lon_max = -55
lat_min = -70
lat_max = -50
ds_mdt_sel = ds_mdt.sel(longitude=slice(lon_min,lon_max), latitude=slice(lat_min,lat_max))

fig = plt.figure(figsize=(10,10))
ax_mdt = plt.axes(projection=projection)
ax_mdt.contourf(ds_mdt_sel['longitude'], ds_mdt_sel['latitude'], ds_mdt_sel['mdt_diff_lat'], cmap='coolwarm', levels= 50, transform=ccrs.PlateCarree())

ax_mdt.coastlines(zorder=100)
ax_mdt.add_feature(cartopy.feature.LAND, color='green', zorder=50)
gl = ax_mdt.gridlines(draw_labels=True, x_inline=False, y_inline=False, linestyle='-.', color='gray')
#ax.set_xticks(np.arange(-66.5,-56,0.5), crs=ccrs.PlateCarree())
#plt.xticks(rotation = 90)
gl.xlabel_style = {'rotation': 45}
#gl.xlabels_bottom = False
gl.xlabels_top = False
gl.ylabels_right = False

In [ ]:
#ADT
ax_adt = fig.add_subplot(gs[1,1], projection=projection)
ax_adt.contourf(ds_adt_sel['longitude'], ds_adt_sel['latitude'], ds_adt_sel['adt'], cmap='coolwarm', levels= 50, transform=ccrs.PlateCarree())

#ADT_diff
ax_adt_diff = fig.add_subplot(gs[1,2], projection=projection)
ax_adt_diff.contourf(ds_adt_sel['longitude'], ds_adt_sel['latitude'], ds_adt_sel['adt_diff'], cmap='coolwarm', levels= 50, transform=ccrs.PlateCarree())

In [ ]:

T_rand = np.random.randint(df_sel['CT'].min()*100, df_sel['CT'].max()*100,100000)/100
S_rand = np.random.randint(df_sel['SA'].min()*100, df_sel['SA'].max()*100,100000)/100
T_rand_diff = np.diff(T_rand)
S_rand_diff = np.diff(S_rand)
rho = gsw.rho(S_rand, T_rand, 0)
bx = (g/rho_0)*(np.diff(rho))

from scipy import ndimage
from matplotlib import ticker
fig, ax = plt.subplots(figsize=(12,6))
#sc = ax.tricontour(S_rand_diff, T_rand_diff, c=np.log10(np.abs(bx)), cmap='coolwarm')
#cb = plt.colorbar(sc, ax=ax)
x = S_rand_diff
y = T_rand_diff
z = np.abs(bx)
#ax.tricontour(x, y, z, levels=14, linewidths=0.5, colors='k')
cntrf = ax.tricontourf(x, y, z, levels=200, cmap="plasma")#, locator=ticker.LogLocator(numticks=20))
#cntr = ax.tricontour(x, y, z, levels=20, colors="black")
cb = fig.colorbar(cntrf, ax=ax)#)

def forward(x):
    return x/10
 
def inverse(x):
    return x*10

xticks = [-4,.0 -2.0, -1.0, -0.5, -0.25, -0.1, 0.1, 0.25, 0.5, 1.0, 2.0, 4.0]
yticks = [-4,.0 -2.0, -1.0, -0.5, -0.25, -0.1, 0.1, 0.25, 0.5, 1.0, 2.0, 4.0]
#ax.set_xscale('function', functions=(forward, inverse))#
ax.set_xscale('symlog', base=2)
ax.set_yscale('symlog', base=2)
ax.set_xticks(xticks, xticks)
ax.set_xticklabels(ax.get_xticks(), rotation = 90)
ax.set_yticks(yticks, yticks)
ax.set_yticklabels(ax.get_yticks(), rotation = 90)
ax.grid()

ax.set(xlabel='$ \Delta S \, [g/kg]$', ylabel='$ \Delta T \, [°C]$')
#ax.xaxis.set_major_locator(MaxNLocator(nbins=6))
#ax.yaxis.set_major_locator(MaxNLocator(nbins=8))
ax.tick_params(direction='out',color='white', grid_linestyle='-.', grid_alpha=0.5)

cb.set_label('$|B_{x}| \. /1000m \. [s^{-2}]$')
plt.savefig

In [ ]:
# User parameters for data test points
n_test = 200  # Number of test data points, tested from 3 to 5000 for subdiv=3

subdiv = 3  # Number of recursive subdivisions of the initial mesh for smooth
            # plots. Values >3 might result in a very high number of triangles
            # for the refine mesh: new triangles numbering = (4**subdiv)*ntri

init_mask_frac = 0.0    # Float > 0. adjusting the proportion of
                        # (invalid) initial triangles which will be masked
                        # out. Enter 0 for no mask.

min_circle_ratio = .01  # Minimum circle ratio - border triangles with circle
                        # ratio below this will be masked if they touch a
                        # border. Suggested value 0.01; use -1 to keep
                        # all triangles.

# meshing with Delaunay triangulation
tri = Triangulation(x, y)
ntri = tri.triangles.shape[0]
# Some invalid data are masked out
mask_init = np.zeros(ntri, dtype=bool)
masked_tri = random_gen.randint(0, ntri, int(ntri * init_mask_frac))
mask_init[masked_tri] = True
tri.set_mask(mask_init)
# masking badly shaped triangles at the border of the triangular mesh.
mask = TriAnalyzer(tri).get_flat_tri_mask(min_circle_ratio)
tri.set_mask(mask)
# refining the data
refiner = UniformTriRefiner(tri)
tri_refi, z_test_refi = refiner.refine_field(z_test, subdiv=subdiv)

In [ ]:
g=9.81
rho_0=1027
T_rand = np.random.randint(df_sel['CT'].min()*100, df_sel['CT'].max()*100,100000)/100
S_rand = np.random.randint(df_sel['SA'].min()*100, df_sel['SA'].max()*100,100000)/100
T_rand_diff = np.diff(T_rand)
S_rand_diff = np.diff(S_rand)
rho = gsw.rho(S_rand, T_rand, 0)
bx = (g/rho_0)*(np.diff(rho)/1000)

df_sim = pd.DataFrame({'T_diff':T_rand_diff, 'S_diff':S_rand_diff, 'Bx':np.abs(bx)})
df_sim_sel = df_sim[df_sim['Bx'] < df_sel['bx_abs'].max()][np.abs(df_sim['T_diff']) < 2.5][np.abs(df_sim['S_diff']) < 0.5]

In [ ]:
g=9.81
rho_0=1027
T_rand = np.random.randint(df_sel['CT'].min()*100, df_sel['CT'].max()*100,100000)/100
S_rand = np.random.randint(df_sel['SA'].min()*100, df_sel['SA'].max()*100,100000)/100
T_rand_diff = np.diff(T_rand)
S_rand_diff = np.diff(S_rand)
rho = gsw.rho(S_rand, T_rand, 0)
bx = (g/rho_0)*(np.diff(rho)/1000)

In [ ]:
min_rho_diff = np.min(df_sel['bx']*1000*(rho_0/g))
max_rho_diff = np.max(df_sel['bx']*1000*(rho_0/g))
rho_diff = np.linspace(min_rho_diff, max_rho_diff, 1000)

In [ ]:

from matplotlib.tri import Triangulation, TriAnalyzer, UniformTriRefiner

min_circle_ratio = .01  # Minimum circle ratio - border triangles with circle
                        # ratio below this will be masked if they touch a
                        # border. Suggested value 0.01; use -1 to keep
                        # all triangles.
subdiv = 3  # Number of recursive subdivisions of the initial mesh for smooth
            # plots. Values >3 might result in a very high number of triangles
            # for the refine mesh: new triangles numbering = (4**subdiv)*ntri

# meshing with Delaunay triangulation
tri = Triangulation(x, y)
ntri = tri.triangles.shape[0]
# masking badly shaped triangles at the border of the triangular mesh.
mask = TriAnalyzer(tri).get_flat_tri_mask(min_circle_ratio)
tri.set_mask(mask)
# refining the data
refiner = UniformTriRefiner(tri)
tri_refi, z_test_refi = refiner.refine_field(z, subdiv=subdiv)

In [ ]:
from matplotlib.ticker import MaxNLocator
import cmocean.cm as cm

df = df_sel_1

ts = df[['CT','SA','lat', 'cruise_id', 'log_bx_abs']]

df = ts.sort_values('CT',ascending=True)

mint = np.min(df['CT'])
maxt = np.max(df['CT'])

mins = np.min(df['SA'])
maxs = np.max(df['SA'])

tempL = np.linspace(mint-1,maxt+1,int(len(df)/6))

salL = np.linspace(mins-0.5,maxs+0.5,int(len(df)/6))

Tg, Sg = np.meshgrid(tempL,salL)
sigma_theta =gsw.sigma0(Sg,Tg)
cnt = np.linspace(sigma_theta.min(),sigma_theta.max(),len(df))

#-----------------------------------------------------------------------------#

fig, (ax0,ax1) = plt.subplots(ncols=2, figsize=(16,8))

#-----------------------------------------------------------------------------#

#ax0
cs = ax0.contour(Sg, Tg, sigma_theta, colors='grey', zorder=1)
cl = plt.clabel(cs, fontsize=14, inline=True, fmt='%.1f')

sc = ax0.scatter(df['SA'], df['CT'], c=df['log_bx_abs'], s=10)
fp = ax0.plot(salL, gsw.CT_freezing(salL, 0, 0), color='royalblue')
cb = plt.colorbar(sc, ax=ax0)

ax0.set(xlabel='Salinity [$g/kg$]', ylabel='Temperature [$°C$]')
ax0.xaxis.set_major_locator(MaxNLocator(nbins=6))
ax0.yaxis.set_major_locator(MaxNLocator(nbins=8))
ax0.tick_params(direction='out')
#cb.set_label('Density [kg m$^{-3}$]')
cb.set_label('Latitude [°N]')

#-----------------------------------------------------------------------------#

#ax1
# cs = ax1.contour(Sg, Tg, sigma_theta, colors='grey', zorder=1)
# cl = plt.clabel(cs, fontsize=14, inline=True, fmt='%.1f')

# sc = ax1.scatter(df['SA'], df['CT'], c=cnt, s=10)
# fp = ax1.plot(salL, gsw.CT_freezing(salL, 0, 0), color='royalblue')
# cb = plt.colorbar(sc, ax=ax0)

# ax1.set(xlabel='Salinity [$g/kg$]', ylabel='Temperature [$°C$]')
# ax1.xaxis.set_major_locator(MaxNLocator(nbins=6))
# ax1.yaxis.set_major_locator(MaxNLocator(nbins=8))
# ax1.tick_params(direction='out')
# cb.set_label('Density [kg m$^{-3}$]')
# #cb.set_label('Latitude [°N]')

#-----------------------------------------------------------------------------#

#ax1
cs = ax1.contour(Sg, Tg, sigma_theta, colors='grey', zorder=1)
cl = plt.clabel(cs, fontsize=14, inline=True, fmt='%.1f')

sc = ax1.scatter(df['SA'], df['CT'], c=pd.factorize(df['cruise_id'])[0], cmap='tab10', s=10)
fp = ax1.plot(salL, gsw.CT_freezing(salL, 0, 0), color='royalblue')
#cb = plt.colorbar(sc, ax=ax1)

ax1.set(xlabel='Salinity [$g/kg$]', ylabel='Temperature [$°C$]')
ax1.xaxis.set_major_locator(MaxNLocator(nbins=6))
ax1.yaxis.set_major_locator(MaxNLocator(nbins=8))
ax1.tick_params(direction='out')
#cb.set_label('Density [kg m$^{-3}$]')
#cb.set_label('Latitude [°N]')
#cb.set_label('Time')
#cb.ax.invert_yaxis()

#-----------------------------------------------------------------------------#

axes = [ax0,ax1]
for ax in axes: 
    freezestr = '\nfreezing point'
    ax.text(33.5, -1.4, 
    freezestr,
    color='blue', 
    #transform=ax.transAxes, 
    fontsize=14,
    verticalalignment='center')

#-----------------------------------------------------------------------------#



# SubantSurWatstr = '\nSubantartic Surface Water'
# ax.text(33.97, 4.0,
#     SubantSurWatstr,
#     color='black',
#     fontsize=14,
#     verticalalignment='center',
#     rotation=35)

# AntSurWatstr = '\nAntarctic Surface Water'
# ax.text(34.1, 1.5,
#     AntSurWatstr,
#     color='black',
#     fontsize=14,
#     verticalalignment='center')

#ConShelfWatstr = '\nContinental Shelf Water'
#ax.text(33.9, -0.1,
 #   ConShelfWatstr,
 #   color='black',
 #   fontsize=14,
 #   verticalalignment='center')

plt.tight_layout()
#fig.savefig('results/analysis/orchestra_2/TS-plots-CTD-casts.png', bbox_inches='tight', format='png')

In [ ]:
#Map
from GG_mthesis.d01_data.processing import rmap 
projection = ccrs.PlateCarree()

fig = plt.figure(figsize=(20,20))
ax = plt.axes(projection=projection)

#bathy
filepath_bath='data/01_raw/ETOPO1_Bed_g_gmt4.grd'
bath = rmap.load(filepath_bath)
lon_min = -67
lon_max = -58
lat_min = -63
lat_max = -55
bath = rmap.select_region(bath, lon_min, lon_max, lat_min, lat_max)
import cmocean.cm as cm
x = bath.x
y = bath.y
z = bath.z
vmin = z.min()
vmax = z.max()
topo_f = ax.contourf(x, y, z, 
        cmap = cm.deep_r,
            vmin = vmin,
            vmax = 0,
        #    norm=MidpointNormalize(
        #        midpoint=-1000,
        #        vmin=vmin,lines
        #        vmax=0
        #     ),
        levels=20
        )
dfs = [df_sel_1, df_sel_2, df_sel_3, df_sel_4, df_sel_5, df_sel_6]
for df_bx in dfs:
    ax.scatter(df_bx['lon'], df_bx['lat'], c='black', s=0.1,transform=ccrs.PlateCarree(), zorder=50,
                #vmax=np.percentile(df_test['bx'], 95), vmin=np.percentile(df_test['bx'], 5)
                )
ax.coastlines(zorder=100)
ax.add_feature(cartopy.feature.LAND, color='green', zorder=50)
gl = ax.gridlines(draw_labels=True, linestyle='-.', color='gray')
#ax.set_xticks(np.arange(-66.5,-56,0.5), crs=ccrs.PlateCarree())
#plt.xticks(rotation = 90)
gl.xlabel_style = {'rotation': 90}
#gl.xlabels_bottom = False
gl.xlabels_top = False
gl.ylabels_right = False
#fig.colorbar(bx)
polygon_df_1.boundary.plot(color='red', linewidth=2, zorder=100, ax=ax, figsize=(20,10))
polygon_df_2.boundary.plot(color='yellow', linewidth=2, zorder=200, ax=ax, figsize=(20,10))
polygon_df_3.boundary.plot(color='lightgreen', linewidth=2, zorder=200, ax=ax, figsize=(20,10))
polygon_df_4.boundary.plot(color='purple', linewidth=2, zorder=200, ax=ax, figsize=(20,10))
polygon_df_5.boundary.plot(color='orange', linewidth=2, zorder=200, ax=ax, figsize=(20,10))
polygon_df_6.boundary.plot(color='orange', linewidth=2, zorder=200, ax=ax, figsize=(20,10))


#ax.plot([-64.4,-64.3], [-55.65, -55.55], c='red')
#ax.plot([-64.3, -58.9], [-55.55, -60.5], c='red')
#ax.plot([-58.9, -59.15], [-60.5, -60.6], c='red')
#ax.plot([-59.15, -64.4], [-60.6,-55.65], c='red')


fronts=rmap.load_fronts('data/01_raw/ACC_FRONTS/PARK/62985.nc')
fronts=rmap.sel_front_region(fronts, lon_min, lon_max, lat_min, lat_max)
s=4
SAF = ax.plot(fronts.loc['SAF']['longitude'], fronts.loc['SAF']['latitude'],transform=ccrs.PlateCarree(), label='SAF_Park', color='purple')#, s=s)
PF = ax.plot(fronts.loc['PF']['longitude'], fronts.loc['PF']['latitude'], transform=ccrs.PlateCarree(), label='PF_Park', color='black')#, s=s)
SACCF = ax.plot(fronts.loc['SACCF']['longitude'], fronts.loc['SACCF']['latitude'],transform=ccrs.PlateCarree(), label='SACCF_Park', color='yellow')#, s=s)
#NB = ax.plot(fronts.loc['NB']['longitude'], fronts.loc['NB']['latitude'],transform=ccrs.PlateCarree(), label='NB_Park', color='red')#, s=s)
#SB = ax.plot(fronts.loc['SB']['longitude'], fronts.loc['SB']['latitude'],transform=ccrs.PlateCarree(), label='SB_Park', color='red')#, s=s)

SAF_p = [SAF_1,SAF_2,SAF_3,SAF_4,SAF_5,SAF_6]
PF_p = [PF_1,PF_2,PF_3,PF_4,PF_5,PF_6]
SACCF_p = [SACCF_1,SACCF_2,SACCF_3,SACCF_4,SACCF_5,SACCF_6]

for i in range(len(SAF_p)):
    ax.scatter(SAF_p[i]['lon'], SAF_p[i]['lat'], c='white', zorder=1000)
    ax.scatter(PF_p[i]['lon'], PF_p[i]['lat'], c='white', zorder=1000)
    ax.scatter(SACCF_p[i]['lon'], SACCF_p[i]['lat'], c='white', zorder=1000)

In [ ]:
df_sel_1_filt[0]['cruise_id'] = df_sel_1_filt[0]['cruise_id'].astype(str)
df_sel_1_filt[1]['cruise_id'] = df_sel_1_filt[1]['cruise_id'].astype(str)
df_sel_1_filt[2]['cruise_id'] = df_sel_1_filt[2]['cruise_id'].astype(str)
s1 = 0
s2 = 1
s3 = 2
df = df_sel_3_filt
x_var = 'lat'
y_var = 'bx'
z_var = 'cruise_id'
fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, figsize=(12,8))
ax0.scatter(df[0][x_var], df[0][y_var], c=df[0][z_var])#, cmap='cividis')


In [ ]:
fig, ax = plt.subplots()
grouped = df_sel_1_filt[0].groupby('cruise_id')
for key, group in grouped:
    group.plot(x='lat', y='bx', ax=ax, kind='line', legend=False)

In [ ]:
df = df_sel_1.sort_values('lat')
bins_size = 0.2
bins = np.arange(round(df_sel['lat'].min(),2),round(df_sel['lat'].max(),2),bins_size)
cats, bins = pd.cut(x = df_sel['lat'], bins = bins, retbins=True)
mean = df_sel['bx'].groupby(cats).count()
std = df_sel['bx'].groupby(cats).std()


fig, ax = plt.subplots(figsize=(12,8))
ax.bar(bins[1:], mean, width=bins_size-0.005, bottom=0.1, align='center', color='black', ecolor='black', capsize=10)
#ax.set_ylabel('Coefficient of Thermal Expansion ($\degree C^{-1}$)')
#ax.set_xticks(df_aux)
#ax.yaxis.grid(True)
sel=0
color = 'red'
ax.plot(SAF_p[sel]['lat'], df['bx_abs'].min(), marker='^', c=color)
ax.plot(PF_p[sel]['lat'], df['bx_abs'].min(), marker='^', c=color)
ax.plot(SACCF_p[sel]['lat'], df['bx_abs'].min(), marker='^', c=color)

# Save the figure and show
plt.tight_layout()

In [ ]:
dfs_base = [df_sel_1, df_sel_2, df_sel_3, df_sel_4, df_sel_5, df_sel_6]
df_base = dfs_base[sel].sort_values('lat')
bins_size = 0.2
bins_base = np.arange(round(df_base['lat'].min(),2),round(df_base['lat'].max(),2),bins_size)
cats_base, bins_base = pd.cut(x = df_base['lat'], bins = bins_base, retbins=True)
mean_base = df_base['bx'].groupby(cats_base).count()
std_base = df_base['bx'].groupby(cats_base).std()

cats, bins = pd.cut(x = dfs[sel]['lat'], bins = bins_base, retbins=True)
mean = dfs[sel]['bx'].groupby(cats).count()
std = dfs[sel]['bx'].groupby(cats).std()
ax0.bar(bins[1:], mean, width=bins_size-0.01, bottom=0.1, align='center', color='black', ecolor='black', capsize=10)

In [ ]:
#ax2 = ax.twinx()
#ax2.bar(bins_base[1:], mean_base, width=bins_size-0.01, alpha=0.5, bottom=0.1, align='center', color='blue', ecolor='black', capsize=10)


#sel = 5
#df = dfs[sel].sort_values('lat')
s = 0
dfs = [df_sel_1_filt[s], df_sel_2_filt[s], df_sel_3_filt[s], df_sel_4_filt[s], df_sel_5_filt[s], df_sel_6_filt[s]]
#bins = np.arange(round(dfs[s]['lat'].min(),2),round(dfs[s]['lat'].max(),2),bins_size)
#cats, bins = pd.cut(x = dfs[sel]['lat'], bins = bins_base, retbins=True)
#mean = dfs[sel]['bx'].groupby(cats).count()
#std = dfs[sel]['bx'].groupby(cats).std()

from matplotlib.gridspec import GridSpec
fig = plt.figure(figsize=(12,8))
gs = GridSpec(3,2)

ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[1,0])
ax3 = fig.add_subplot(gs[1,1])
ax4 = fig.add_subplot(gs[2,0])
ax5 = fig.add_subplot(gs[2,1])
axes = [ax0, ax1, ax2, ax3, ax4, ax5]

def plot_bars(sel, ax, bin_size):
    bins_base = np.arange(round(dfs[sel]['lat'].min(),2),round(dfs[sel]['lat'].max(),2),bin_size)
    cats, bins = pd.cut(x = dfs[sel]['lat'], bins = bins_base, retbins=True)
    mean = dfs[sel]['bx'].groupby(cats).count()
    #std = dfs[sel]['bx'].groupby(cats).std()
    ax.bar(bins[1:], mean, width=bin_size-0.02, bottom=0.1, align='center', color='black', ecolor='black', capsize=10)

def front_lat(sel, ax, color='red', marker='^', markersize=10, y_value=-0.5):
    ax.plot(SAF_p[i]['lat'], y_value, marker=marker, markersize=markersize, c=color)
    ax.plot(PF_p[i]['lat'], y_value, marker=marker, markersize=markersize, c=color)
    ax.plot(SACCF_p[i]['lat'], y_value, marker=marker, markersize=markersize, c=color)
    
for i in range(len(axes)):
    plot_bars(i, axes[i], bin_size=0.2)
    front_lat(i, axes[i])
    axes[i].invert_xaxis()


# Save the figure and show
plt.tight_layout()

In [ ]:
s = 0
dfs = [df_sel_1_filt[s], df_sel_2_filt[s], df_sel_3_filt[s], df_sel_4_filt[s], df_sel_5_filt[s], df_sel_6_filt[s]]
sel = 5
df = dfs[sel].sort_values('lat')
rolling = 20
df_mean = df.rolling(rolling).mean()
df_std = df.sort_values('lat').rolling(rolling).std()
df_std_1 = df_mean + df_std
df_std_2  = df_mean - df_std
fig, ax = plt.subplots()
SAF_p = [SAF_1,SAF_2,SAF_3,SAF_4,SAF_5,SAF_6]
PF_p = [PF_1,PF_2,PF_3,PF_4,PF_5,PF_6]
SACCF_p = [SACCF_1,SACCF_2,SACCF_3,SACCF_4,SACCF_5,SACCF_6]
ax.plot(df['lat'], df_mean['bx'], c='black')
ax.plot(df['lat'], df_std_1['bx'], c='red', linestyle='-.')
ax.plot(df['lat'], df_std_2['bx'], c='red', linestyle='-.')
ax.plot(SAF_p[sel]['lat'], df_std_2['bx'].min(), marker='^', c='black')
ax.plot(PF_p[sel]['lat'], df_std_2['bx'].min(), marker='^', c='black')
ax.plot(SACCF_p[sel]['lat'], df_std_2['bx'].min(), marker='^', c='black')

SAF_p = [SAF_1,SAF_2,SAF_3,SAF_4,SAF_5,SAF_6]
PF_p = [PF_1,PF_2,PF_3,PF_4,PF_5,PF_6]
SACCF_p = [SACCF_1,SACCF_2,SACCF_3,SACCF_4,SACCF_5,SACCF_6]

In [ ]:
s1 = 0
s2 = 1
s3 = 2
df = df_sel_3_filt
fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, figsize=(12,6))

ax0.scatter(df[s1]['S_diff'], df[s1]['T_diff'], c=df[s1]['bx_abs'])
ax0.set_xlim([-0.5,0.5])
ax1.scatter(df[s2]['S_diff'], df[s2]['T_diff'], c=df[s2]['lat'])
ax2.scatter(df[s3]['S_diff'], df[s3]['T_diff'], c=df[s3]['lat'])

In [ ]:
#ax2 = ax.twinx()
#ax2.bar(bins_base[1:], mean_base, width=bins_size-0.01, alpha=0.5, bottom=0.1, align='center', color='blue', ecolor='black', capsize=10)

#df = dfs[sel].sort_values('lat')
s = 0
dfs = [df_sel_1_filt[s], df_sel_2_filt[s], df_sel_3_filt[s], df_sel_4_filt[s], df_sel_5_filt[s], df_sel_6_filt[s]]
#bins = np.arange(round(dfs[s]['lat'].min(),2),round(dfs[s]['lat'].max(),2),bins_size)
#cats, bins = pd.cut(x = dfs[sel]['lat'], bins = bins_base, retbins=True)
#mean = dfs[sel]['bx'].groupby(cats).count()
#std = dfs[sel]['bx'].groupby(cats).std()

from matplotlib.gridspec import GridSpec
fig = plt.figure(figsize=(12,8))
gs = GridSpec(3,2)

ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[1,0])
ax3 = fig.add_subplot(gs[1,1])
ax4 = fig.add_subplot(gs[2,0])
ax5 = fig.add_subplot(gs[2,1])
axes = [ax0, ax1, ax2, ax3, ax4, ax5]

def plot_bars(df, ax, var='lat', bin_size=0.2):
    bins_base = np.arange(round(df[var].min(),2),round(df[var].max(),2),bin_size)
    #df = df.sort_values(var)
    cats, bins = pd.cut(x = df[var], bins = bins_base, retbins=True)
    count = df['bx'].groupby(cats).count()
    #std = dfs[sel]['bx'].groupby(cats).std()
    ax.bar(bins[:-1], count, width=bin_size-0.02, bottom=0.1, align='center', color='black', ecolor='black', capsize=10)

SAF_p = [SAF_1,SAF_2,SAF_3,SAF_4,SAF_5,SAF_6]
PF_p = [PF_1,PF_2,PF_3,PF_4,PF_5,PF_6]
SACCF_p = [SACCF_1,SACCF_2,SACCF_3,SACCF_4,SACCF_5,SACCF_6]

def front_lat(SAF, PF, SACCF, ax, color='red', marker='^', markersize=10, y_value=-0.5):
    ax.plot(SAF['lat'], y_value, marker=marker, markersize=markersize, c=color)
    ax.plot(PF['lat'], y_value, marker=marker, markersize=markersize, c=color)
    ax.plot(SACCF['lat'], y_value, marker=marker, markersize=markersize, c=color)

for i in range(len(axes)):
    plot_bars(dfs[i], axes[i], bin_size=0.2)
    front_lat(SAF_p[i], PF_p[i], SACCF_p[i], axes[i])
    axes[i].invert_xaxis()


# Save the figure and show
plt.tight_layout()

In [ ]:
from scipy import ndimage
from matplotlib import ticker
import matplotlib
fig, ax = plt.subplots(figsize=(12,6))
#sc = ax.tricontour(S_rand_diff, T_rand_diff, c=np.log10(np.abs(bx)), cmap='coolwarm')
#cb = plt.colorbar(sc, ax=ax)


x = df_sim['S_diff']
y = df_sim['T_diff']
z = np.log10(np.abs(df_sim['bx']))

#ax.tricontour(x, y, z, levels=14, linewidths=0.5, colors='k')
cntrf = ax.tricontourf(x, y, z, levels=100, cmap="plasma",
                       vmax=-6
                       )#, norm=matplotlib.colors.LogNorm())#, locator=ticker.LogLocator(numticks=20))
#cntr = ax.tricontour(x, y, z, levels=10, colors="black")
cb = fig.colorbar(cntrf, ax=ax)#)


def forward(x):
    return x/10
 
def inverse(x):
    return x*10
#ax.set_xscale('function', functions=(forward, inverse))#
#ax.set_xscale('symlog', base=2)
#ax.set_yscale('symlog', base=2)

xticks = [-4,.0 -2.0, -1.0, -0.5, -0.25, -0.1, 0.1, 0.25, 0.5, 1.0, 2.0, 4.0]
yticks = [-4,.0 -2.0, -1.0, -0.5, -0.25, -0.1, 0.1, 0.25, 0.5, 1.0, 2.0, 4.0]
#ax.set_xticks(xticks, xticks)
#ax.set_xticklabels(ax.get_xticks(), rotation = 90)
#ax.set_yticks(yticks, yticks)
#ax.set_yticklabels(ax.get_yticks(), rotation = 90)
ax.grid()

ax.set(xlabel='$ \Delta S \, [g/kg]$', ylabel='$ \Delta T \, [°C]$')
#ax.xaxis.set_major_locator(MaxNLocator(nbins=6))
#ax.yaxis.set_major_locator(MaxNLocator(nbins=8))
ax.tick_params(direction='out',color='white', grid_linestyle='-.', grid_alpha=0.5)

cb.set_label('$|B_{x}| \: / \: 1km \: [s^{-2}]$')

ax.set_xlim([-0.2,0.2])
#plt.savefig(r'results/final/bx_relation.png', bbox_inches='tight', format='png')

In [ ]:
x = df_sim['S_diff']
y = df_sim['T_diff']
z = np.abs(df_sim['bx'])

xi = np.arange(-0.1, 0.1, 0.001)
yi = np.arange(-1, 1.2, 0.001)
from scipy.interpolate import griddata
zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='linear')
df = df_sel_1_filt[1]
fig, ax1 = plt.subplots(figsize=(12,8))
#ax1.contour(xi, yi, zi, levels=100, linewidths=0.5, colors='k')
cntr1 = ax1.contourf(xi, yi, zi, levels=100, cmap="RdBu_r", vmax=5E-7)
ax1.scatter(df['S_diff'], df['T_diff'], c='black')
fig.colorbar(cntr1, ax=ax1)

In [ ]:
x = df_sim['S_diff']
y = df_sim['T_diff']
z = np.log10(np.abs(df_sim['bx']))

xi = np.arange(-0.2, 0.15, 0.001)
yi = np.arange(-1.6, 1.1, 0.001)
from scipy.interpolate import griddata
zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='linear')
df = df_sel_2_filt[0]
from matplotlib import colors
fig, ax1 = plt.subplots(figsize=(12,8))
divnorm=colors.TwoSlopeNorm(vmin=-10, vcenter=-6.5, vmax=-5.5)
#ax1.contour(xi, yi, zi, levels=100, linewidths=0.5, colors='k')
cntr1 = ax1.contourf(xi, yi, zi, levels=100, cmap="RdBu_r", norm=divnorm)
ax1.scatter(df['S_diff'], df['T_diff'], c='black')
fig.colorbar(cntr1, ax=ax1)

In [ ]:
distance = 1000
g = 9.81 #m s-2
rho_0 = 1027 #kg m-3
T1 = 10
T2 = 5
S1 = 33.6
S2 = 34.2
rho1 = gsw.rho(S1, T1, 0)
rho2 = gsw.rho(S2, T2, 0)
bx = (g/rho_0)*(rho2-rho1)/1000

bx

In [ ]:
T_rand = np.random.randint(df_sel['CT'].min()*10000, df_sel['CT'].max()*10000,10000000)/10000
S_rand = np.random.randint(df_sel['SA'].min()*10000, df_sel['SA'].max()*10000,10000000)/10000
df_sim = pd.DataFrame({'T':T_rand,'S':S_rand})
df_sim['T_diff'] = df_sim['T'].diff()
df_sim['S_diff'] = df_sim['S'].diff()
df_sim['rho'] = gsw.rho(df_sim['S'].values, df_sim['T'].values, 0)
g=9.81
rho_0=1027
df_sim['bx'] = (g/rho_0)*(df_sim['rho'].diff()/1000)
df_sim = df_sim[np.abs(df_sim['T_diff']) < np.abs(df_sel['T_diff']).max()][np.abs(df_sim['S_diff']) < np.abs(df_sel['S_diff']).max()]
df_sim_2 = df_sim[np.abs(df_sim['T_diff']) < 1E-6][np.abs(df_sim['S_diff']) < np.abs(df_sel['S_diff']).max()]

In [ ]:
#df_sim_2 = df_sim[np.abs(df_sim['bx']) < np.percentile(df_sel['bx_abs'], 90)]
fig, ax = plt.subplots(figsize=(12,6))
df_sim_2 = df_sim[np.abs(df_sim['bx']) < 1E-6]
sc = ax.scatter(df_sim_2['S_diff'], df_sim_2['T_diff'], c=np.abs(df_sim_2['bx']))
df = df_sel_1_filt[1]
ax.scatter(df['S_diff'], df['T_diff'], c=df['bx_abs'], cmap='coolwarm')
ax.set_xlim(-0.1,0.1)
ax.set_ylim(-0.2,0.2)
fig.colorbar(sc, ax=ax)

In [ ]:
df = dfs[sel].sort_values('lat')
s = 0
dfs = [df_sel_1_filt[s], df_sel_2_filt[s], df_sel_3_filt[s], df_sel_4_filt[s], df_sel_5_filt[s], df_sel_6_filt[s]]

bins_base = np.arange(round(dfs[sel]['lat'].min(),2),round(dfs[sel]['lat'].max(),2),bin_size)
cats, bins = pd.cut(x = dfs[sel]['lat'], bins = bins_base, retbins=True)
mean = dfs[sel]['bx'].groupby(cats).count()

In [ ]:
bin_size=0.1
df1 = df_sel_3_filt[0]
df2 = df_sel_3#df_sel_5_filt[1].append(df_sel_5_filt[2])
bins_base = np.arange(round(df_sel_5['lat'].min(),2),round(df_sel_5['lat'].max(),2),bin_size)
#df = df.sort_values(var)
cats1, bins1 = pd.cut(x = df1['lat'], bins = bins_base, retbins=True)
count1 = df1['bx'].groupby(cats1).count()

cats2, bins2 = pd.cut(x = df2['lat'], bins = bins_base, retbins=True)
count2 = df2['bx'].groupby(cats2).count()

fig, ax = plt.subplots()
ratio = (count1/count2)
ax1 = ax.twinx()

rolling = ratio.rolling(2).mean()

ax.bar(bins1[:-1], ratio, width=bin_size-0.02, bottom=0.1, align='center', color='black', ecolor='black', capsize=10)
ax1.plot(bins1[:-1], rolling, c='red', linewidth=4)
ax.invert_xaxis()

In [ ]:
def dens_plot_2d(df, x_var, y_var, ax, nbins=100, cbar_loc='bottom'):
    from scipy.stats import kde
    x = df[x_var]
    y = df[y_var]
    nbins = nbins
    k = kde.gaussian_kde([x,y])
    xi, yi = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
    zi = k(np.vstack([xi.flatten(), yi.flatten()]))

    pc = ax.pcolormesh(xi, yi, zi.reshape(xi.shape), shading='auto', cmap='coolwarm')
    #fig.colorbar(pc, location=cbar_loc, pad=0.01, ax=ax)
 
from matplotlib.gridspec import GridSpec
fig = plt.figure(constrained_layout=True, figsize=(16,8))
gs = GridSpec(3, 2, figure=fig) #, hspace=0.2,wspace=0.1)
    
ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[1,0], sharex=ax0)
ax3 = fig.add_subplot(gs[1,1], sharex=ax1)
ax4 = fig.add_subplot(gs[2,0], sharex=ax0)
ax5 = fig.add_subplot(gs[2,1], sharex=ax1)

section = 0
axes = [ax0, ax1, ax2, ax3, ax4, ax5]
#dfs = [df_sel_1, df_sel_2, df_sel_3, df_sel_4, df_sel_5, df_sel_6]
dfs = [ df_sel_1_filt[section], df_sel_2_filt[section], df_sel_3_filt[section], df_sel_4_filt[section], df_sel_5_filt[section], df_sel_6_filt[section]]

SAF_p = [SAF_1,SAF_2,SAF_3,SAF_4,SAF_5,SAF_6]
PF_p = [PF_1,PF_2,PF_3,PF_4,PF_5,PF_6]
SACCF_p = [SACCF_1,SACCF_2,SACCF_3,SACCF_4,SACCF_5,SACCF_6]


from scipy.interpolate import griddata

x_var = 'mdt'
y_var = 'bx_abs'
z_var = 'mdt_diff_lat'
for i in range(len(axes)):
    #dens_plot_2d(dfs[i], x_var, y_var, ax=axes[i])
    sc = axes[i].scatter(dfs[i][x_var], dfs[i][y_var], c=dfs[i][z_var], cmap='coolwarm')
    #lp = axes[i].plot(dfs[i].sort_values(x_var)[x_var], dfs[i].sort_values(x_var)[y_var], c='black', alpha=0.7)
    marker_color = 'red'
    #df = dfs[i].sort_values('lat').rolling(20).mean()
    #df2 = dfs[i].sort_values('lat').rolling(50).median()
    #lat_grid = np.arange(dfs[i]['lat'].min(),dfs[i]['lat'].max(),0.02)
    #bx_grid = griddata(points=dfs[i]['lat'].values, values=dfs[i][y_var].values, xi=lat_grid)
    #rol = pd.DataFrame({'lat':lat_grid, 'bx':bx_grid}).rolling(20, center=True).median()
    #tax = axes[i].twinx()
    #lp_rolling = tax.plot(rol['lat'], rol['bx'], c='black')
    #lp_rolling_median = tax.plot(df2[x_var], df2[y_var], c='green')
    
    axes[i].scatter(-0.1, 1.01*dfs[i][y_var].min(), marker='^', c=marker_color, zorder=500)
    axes[i].scatter(-0.58, 1.01*dfs[i][y_var].min(), marker='^', c=marker_color, zorder=500)
    axes[i].scatter(-1.0, 1.01*dfs[i][y_var].min(), marker='^', c=marker_color, zorder=500)
    axes[i].invert_xaxis()
    axes[i].set_yscale('log')
    cb = fig.colorbar(sc, location='bottom', pad=0.01, ax=axes[i])
    cb.ax.invert_xaxis()

In [ ]:
ds_mdt_sel = ds_mdt.sel(longitude=slice(-65.3,-59.0), latitude=slice(-63,-55.0))

def interpolate(df, z_var, x_var='lon', y_var='lat', ssh_prod = 'mdt', grid_size=0.05):
    x = df[x_var]
    y = df[y_var]
    z = df[z_var]
    x_grid = np.arange(x.min(), x.max(), grid_size)
    y_grid = np.arange(y.min(), y.max(), grid_size)
    x_grid, y_grid = np.meshgrid(x_grid,y_grid)
    xi = (x_grid, y_grid)

    points =  df[[x_var,y_var]].to_numpy() #np.array([df2['lon'].to_numpy(), df2['lat'].to_numpy()])
    values = (z.to_numpy())

    from scipy import interpolate
    z_interpol = interpolate.griddata((x, y), values, xi, method='linear')
    return x_grid, y_grid, z_interpol

fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(projection=ccrs.PlateCarree())

ax.contourf(ds_mdt_sel['longitude'], ds_mdt_sel['latitude'], ds_mdt_sel['mdt'], cmap='coolwarm', levels= 50)
var = 'log_bx_abs'
vmin = df_sel_1[var].min()
vmax = df_sel_1[var].max()
x_grid_1, y_grid_1, bx_grid_1 = interpolate(df_sel_1, var)
x_grid_2, y_grid_2, bx_grid_2 = interpolate(df_sel_2, var)
x_grid_3, y_grid_3, bx_grid_3 = interpolate(df_sel_3, var)
x_grid_4, y_grid_4, bx_grid_4 = interpolate(df_sel_4, var)
x_grid_5, y_grid_5, bx_grid_5 = interpolate(df_sel_5, var)
x_grid_6, y_grid_6, bx_grid_6 = interpolate(df_sel_6, var)

ax.contourf(x_grid_1, y_grid_1, bx_grid_1, levels=40, cmap='coolwarm', vmin=vmin, vmax=vmax)
ax.contourf(x_grid_2, y_grid_2, bx_grid_2, levels=40, cmap='coolwarm', vmin=vmin, vmax=vmax)
ax.contourf(x_grid_3, y_grid_3, bx_grid_3, levels=40, cmap='coolwarm', vmin=vmin, vmax=vmax)
ax.contourf(x_grid_4, y_grid_4, bx_grid_4, levels=40, cmap='coolwarm', vmin=vmin, vmax=vmax)
ax.contourf(x_grid_5, y_grid_5, bx_grid_5, levels=40, cmap='coolwarm', vmin=vmin, vmax=vmax)
ax.contourf(x_grid_6, y_grid_6, bx_grid_6, levels=40, cmap='coolwarm', vmin=vmin, vmax=vmax)

#ax.contourf(ds_mdt_sel['longitude'], ds_mdt_sel['latitude'], ds_mdt_sel['mdt'], colors='black', levels= 10)

lon_min=-65.3
lon_max=-59.0
lat_min=-63.0
lat_max=-55.0

fronts=rmap.load_fronts('data/01_raw/ACC_FRONTS/PARK/62985.nc')
fronts=rmap.sel_front_region(fronts, lon_min, lon_max, lat_min, lat_max)
s=4
SAF = ax.plot(fronts.loc['SAF']['longitude'], fronts.loc['SAF']['latitude'],transform=ccrs.PlateCarree(), label='SAF_Park', color='purple')#, s=s)
PF = ax.plot(fronts.loc['PF']['longitude'], fronts.loc['PF']['latitude'], transform=ccrs.PlateCarree(), label='PF_Park', color='black')#, s=s)
SACCF = ax.plot(fronts.loc['SACCF']['longitude'], fronts.loc['SACCF']['latitude'],transform=ccrs.PlateCarree(), label='SACCF_Park', color='yellow')#, s=s)
#NB = ax.plot(fronts.loc['NB']['longitude'], fronts.loc['NB']['latitude'],transform=ccrs.PlateCarree(), label='NB_Park', color='red')#, s=s)
#SB = ax.plot(fronts.loc['SB']['longitude'], fronts.loc['SB']['latitude'],transform=ccrs.PlateCarree(), label='SB_Park', color='red')#, s=s)

In [ ]:
def rollBy(what, basis, window, func):
    def applyToWindow(val):
        chunk = what[(val<=basis) & (basis<val+window)]
        return func(chunk)
    return basis.apply(applyToWindow)

import statistics

rollBy(df_sel_1.bx, df_sel_1.lat, 0.1, statistics.mean)f

In [ ]:
def plot_bars_ratio(df, df_base, ax, var='lat', bin_size=0.1, rolling=4):
    df1 = df
    df2 = df_base
    bins_base = np.arange(round(df2[var].min(),2),round(df2[var].max(),2),bin_size)
    cats1, bins1 = pd.cut(x = df1['lat'], bins = bins_base, retbins=True)
    count1 = df1['bx_abs'].groupby(cats1).count()
    cats2, bins2 = pd.cut(x = df2['lat'], bins = bins_base, retbins=True)
    count2 = df2['bx_abs'].groupby(cats2).count()
    ratio = (count1/count2)*100
    rolling = ratio.rolling(rolling, center=True, closed='both').mean()
    ax_aux = ax.twinx()
    ax.bar(bins1[:-1], ratio, width=bin_size-0.02, bottom=0.01, align='center', color='black', ecolor='black', capsize=10)
    ax_aux.plot(bins1[:-1], rolling, c='red', linewidth=4)

SAF_p = [SAF_1,SAF_2,SAF_3,SAF_4,SAF_5,SAF_6]
PF_p = [PF_1,PF_2,PF_3,PF_4,PF_5,PF_6]
SACCF_p = [SACCF_1,SACCF_2,SACCF_3,SACCF_4,SACCF_5,SACCF_6]

def front_lat(SAF, PF, SACCF, ax, color='red', marker='^', markersize=10, y_value=-0.01):
    ax.plot(SAF['lat'], y_value, marker=marker, markersize=markersize, c='purple')
    ax.plot(PF['lat'], y_value, marker=marker, markersize=markersize, c='red')
    ax.plot(SACCF['lat'], y_value, marker=marker, markersize=markersize, c='green')

from matplotlib.gridspec import GridSpec
fig = plt.figure(figsize=(16,8), tight_layout=True)
gs = GridSpec(3,2)

ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1], sharex=ax0)
ax2 = fig.add_subplot(gs[1,0], sharex=ax0)
ax3 = fig.add_subplot(gs[1,1], sharex=ax0)
ax4 = fig.add_subplot(gs[2,0], sharex=ax0)
ax5 = fig.add_subplot(gs[2,1], sharex=ax0)
axes = [ax0, ax1, ax2, ax3, ax4, ax5]

s = 0
dfs_filt = [df_sel_1_filt[s], df_sel_2_filt[s], df_sel_3_filt[s], df_sel_4_filt[s], df_sel_5_filt[s], df_sel_6_filt[s]]
dfs_base = [df_sel_1, df_sel_2, df_sel_3, df_sel_4, df_sel_5, df_sel_6]
for i in range(len(axes)):
    plot_bars_ratio(dfs_filt[i], dfs_base[i], axes[i], bin_size=0.1, rolling=4)
    front_lat(SAF_p[i], PF_p[i], SACCF_p[i], axes[i], y_value=-0.02)
    
ax0.invert_xaxis()
ax4.set_xlabel('$Latitude \: [°]$', fontsize=11)
ax5.set_xlabel('$Latitude \: [°]$', fontsize=11)

ax0.set_ylabel('% of outliers per 0.2° bin', fontsize=10)
ax2.set_ylabel('% of outliers per 0.2° bin', fontsize=10)
ax4.set_ylabel('% of outliers per 0.2° bin', fontsize=10)

A = ax0.text(0.01, 0.95, 'T1', fontsize=12, fontweight='bold', ha='left', va='top', transform = ax0.transAxes)

B = ax1.text(0.01, 0.95, 'T2', fontsize=12, fontweight='bold', ha='left', va='top', transform = ax1.transAxes)

C = ax2.text(0.01, 0.95, 'T3', fontsize=12, fontweight='bold', ha='left', va='top', transform = ax2.transAxes)

D = ax3.text(0.01, 0.95, 'T4', fontsize=12, fontweight='bold', ha='left', va='top', transform = ax3.transAxes)

E = ax4.text(0.01, 0.95, 'T5', fontsize=12, fontweight='bold', ha='left', va='top', transform = ax4.transAxes)

F = ax5.text(0.01, 0.95, 'T6', fontsize=12, fontweight='bold', ha='left', va='top', transform = ax5.transAxes)



plt.savefig(r'results/final/bar_plot_bins.png', bbox_inches='tight', format='png')


In [ ]:
####### IMPORTANT ############

def plot_bars_ratio(df, df_base, ax, ax_twin, var='lat', bin_size=0.1, rolling_value=4):
    df1 = df
    df2 = df_base
    bins_base = np.arange(round(df2[var].min(),2),round(df2[var].max(),2),bin_size)
    cats1, bins1 = pd.cut(x = df1['lat'], bins = bins_base, retbins=True)
    count1 = df1['bx_abs'].groupby(cats1).count()
    cats2, bins2 = pd.cut(x = df2['lat'], bins = bins_base, retbins=True)
    count2 = df2['bx_abs'].groupby(cats2).count()
    ratio = (count1/count2)*100
    rolling = ratio.rolling(rolling_value, center=True, closed='both').median()
    mdt = df2['mdt_diff_lat'].groupby(cats2).median()
    #rolling2 = ratio.rolling(rolling_value, center=True, closed='both').median()
    
    ax.bar(bins1[:-1], ratio, width=bin_size-0.02, bottom=0.01, align='center', color='black', ecolor='black', capsize=10)
    #ax.plot(bins1[:-1], ratio, color='blue')
    ax.plot(bins1[:-1], rolling, c='green', linewidth=4)
    #ax_aux.plot(bins1[:-1], rolling2, c='blue', linewidth=4)
    ax_twin.plot(bins1[:-1], mdt, c='red', linewidth=4)

SAF_p = [SAF_1,SAF_2,SAF_3,SAF_4,SAF_5,SAF_6]
PF_p = [PF_1,PF_2,PF_3,PF_4,PF_5,PF_6]
SACCF_p = [SACCF_1,SACCF_2,SACCF_3,SACCF_4,SACCF_5,SACCF_6]

def front_lat(SAF, PF, SACCF, ax, color='red', marker='^', markersize=12, y_value=-0.005):
    ax.plot(SAF['lat'], y_value, marker=marker, markersize=markersize, c='blue')
    ax.plot(PF['lat'], y_value, marker=marker, markersize=markersize, c='red')
    ax.plot(SACCF['lat'], y_value, marker=marker, markersize=markersize, c='green')

from matplotlib.gridspec import GridSpec
fig = plt.figure(figsize=(16,8), tight_layout=True)
gs = GridSpec(3,2)

ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1], sharex=ax0)
ax2 = fig.add_subplot(gs[1,0], sharex=ax0)
ax3 = fig.add_subplot(gs[1,1], sharex=ax0)
ax4 = fig.add_subplot(gs[2,0], sharex=ax0)
ax5 = fig.add_subplot(gs[2,1], sharex=ax0)
axes = [ax0, ax1, ax2, ax3, ax4, ax5]

ax_twin0 = ax0.twinx()
ax_twin1 = ax1.twinx()
ax_twin2 = ax2.twinx()
ax_twin3 = ax3.twinx()
ax_twin4 = ax4.twinx()
ax_twin5 = ax5.twinx()
twin_axes = [ax_twin0, ax_twin1, ax_twin2, ax_twin3, ax_twin4, ax_twin5]

mdts = [mdt_1, mdt_2, mdt_3, mdt_4, mdt_5, mdt_6]


s = 0
dfs_filt = [df_sel_1_filt[s], df_sel_2_filt[s], df_sel_3_filt[s], df_sel_4_filt[s], df_sel_5_filt[s], df_sel_6_filt[s]]
dfs_base = [df_sel_1, df_sel_2, df_sel_3, df_sel_4, df_sel_5, df_sel_6]
for i in range(len(axes)):
    plot_bars_ratio(dfs_filt[i], dfs_base[i], axes[i], twin_axes[i], bin_size=0.1, rolling_value=3)
    front_lat(SAF_p[i], PF_p[i], SACCF_p[i], axes[i], y_value=-0.02)
    
ax0.invert_xaxis()
ax4.set_xlabel('Latitude [°]', fontsize=12, fontweight='bold')
ax5.set_xlabel('Latitude [°]', fontsize=12, fontweight='bold')

ax0.set_ylabel('% of outliers per 0.1° bin', fontsize=10, fontweight='bold')
ax2.set_ylabel('% of outliers per 0.1° bin', fontsize=10, fontweight='bold')
ax4.set_ylabel('% of outliers per 0.1° bin', fontsize=10, fontweight='bold')

ax_twin1.set_ylabel('0.5° rolling mean', fontsize=10, fontweight='bold')
ax_twin3.set_ylabel('0.5° rolling mean', fontsize=10, fontweight='bold')
ax_twin5.set_ylabel('0.5° rolling mean', fontsize=10, fontweight='bold')

A = ax0.text(0.01, 0.95, 'T1', fontsize=14, fontweight='bold', ha='left', va='top', transform = ax0.transAxes)

B = ax1.text(0.01, 0.95, 'T2', fontsize=14, fontweight='bold', ha='left', va='top', transform = ax1.transAxes)

C = ax2.text(0.01, 0.95, 'T3', fontsize=14, fontweight='bold', ha='left', va='top', transform = ax2.transAxes)

D = ax3.text(0.01, 0.95, 'T4', fontsize=14, fontweight='bold', ha='left', va='top', transform = ax3.transAxes)

E = ax4.text(0.01, 0.95, 'T5', fontsize=14, fontweight='bold', ha='left', va='top', transform = ax4.transAxes)

F = ax5.text(0.01, 0.95, 'T6', fontsize=14, fontweight='bold', ha='left', va='top', transform = ax5.transAxes)


#plt.savefig(r'results/final/bar_plot_bins_rolling_mean.png', bbox_inches='tight', format='png')


In [ ]:
def plot_bars_ratio(df, ax, ax_twin, var='lat', bin_size=0.1, rolling_value=2):
    bins_base = np.arange(round(df[var].min(),2),round(df[var].max(),2),bin_size)
    cats1, bins1 = pd.cut(x = df['lat'], bins = bins_base, retbins=True)
    count1 = df['bx_abs'].groupby(cats1).mean()
    count2 = df['bx_abs'].groupby(cats1).std()
    rolling2 = count1.rolling(rolling_value, center=True, closed='both').std()
    y1 = count1+count2
    y2 = count1-count2
    #ax.bar(bins1[:-1], count1, width=bin_size-0.02, bottom=0.01, align='center', color='black', ecolor='black', capsize=10)
    ax.plot(bins1[:-1], count2, color='black')
    #ax.plot(bins1[:-1], y1, color='red', linestyle='-.')
    #ax.plot(bins1[:-1], y2, color='red', linestyle='-.')
    lower_err = (y1 - count2).abs() 
    upper_err = (y1 - count2).abs()
    yerr = [lower_err, upper_err]
    #ax.errorbar(bins1[:-1], count2, yerr=yerr, capsize=3)
    #ax_twin.plot(bins1[:-1], count2, color='red')
    #ax_twin.plot(bins1[:-1], rolling2, c='red', linewidth=4)

SAF_p = [SAF_1,SAF_2,SAF_3,SAF_4,SAF_5,SAF_6]
PF_p = [PF_1,PF_2,PF_3,PF_4,PF_5,PF_6]
SACCF_p = [SACCF_1,SACCF_2,SACCF_3,SACCF_4,SACCF_5,SACCF_6]

def front_lat(SAF, PF, SACCF, ax, color='red', marker='^', markersize=12, y_value=-0.005):
    ax.plot(SAF['lat'], y_value, marker=marker, markersize=markersize, c='blue')
    ax.plot(PF['lat'], y_value, marker=marker, markersize=markersize, c='red')
    ax.plot(SACCF['lat'], y_value, marker=marker, markersize=markersize, c='green')

from matplotlib.gridspec import GridSpec
fig = plt.figure(figsize=(16,8), tight_layout=True)
gs = GridSpec(3,2)

ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1], sharex=ax0)
ax2 = fig.add_subplot(gs[1,0], sharex=ax0)
ax3 = fig.add_subplot(gs[1,1], sharex=ax0)
ax4 = fig.add_subplot(gs[2,0], sharex=ax0)
ax5 = fig.add_subplot(gs[2,1], sharex=ax0)
axes = [ax0, ax1, ax2, ax3, ax4, ax5]

ax_twin0 = ax0.twinx()
ax_twin1 = ax1.twinx()
ax_twin2 = ax2.twinx()
ax_twin3 = ax3.twinx()
ax_twin4 = ax4.twinx()
ax_twin5 = ax5.twinx()
twin_axes = [ax_twin0, ax_twin1, ax_twin2, ax_twin3, ax_twin4, ax_twin5]
mdts = [mdt_1, mdt_2, mdt_3, mdt_4, mdt_5, mdt_6]
s = 0
dfs_base = [df_sel_1, df_sel_2, df_sel_3, df_sel_4, df_sel_5, df_sel_6]
for i in range(len(axes)):
    plot_bars_ratio(dfs_base[i], axes[i], twin_axes[i], bin_size=0.1)
    front_lat(SAF_p[i], PF_p[i], SACCF_p[i], axes[i], y_value=0)
    twin_axes[i].plot(mdts[i]['latitude'], mdts[i], color='pink')
    
ax0.invert_xaxis()
ax4.set_xlabel('Latitude [°]', fontsize=12, fontweight='bold')
ax5.set_xlabel('Latitude [°]', fontsize=12, fontweight='bold')

ax0.set_ylabel('% of outliers per 0.1° bin', fontsize=10, fontweight='bold')
ax2.set_ylabel('% of outliers per 0.1° bin', fontsize=10, fontweight='bold')
ax4.set_ylabel('% of outliers per 0.1° bin', fontsize=10, fontweight='bold')

ax_twin1.set_ylabel('0.5° rolling mean', fontsize=10, fontweight='bold')
ax_twin3.set_ylabel('0.5° rolling mean', fontsize=10, fontweight='bold')
ax_twin5.set_ylabel('0.5° rolling mean', fontsize=10, fontweight='bold')

A = ax0.text(0.01, 0.95, 'T1', fontsize=14, fontweight='bold', ha='left', va='top', transform = ax0.transAxes)

B = ax1.text(0.01, 0.95, 'T2', fontsize=14, fontweight='bold', ha='left', va='top', transform = ax1.transAxes)

C = ax2.text(0.01, 0.95, 'T3', fontsize=14, fontweight='bold', ha='left', va='top', transform = ax2.transAxes)

D = ax3.text(0.01, 0.95, 'T4', fontsize=14, fontweight='bold', ha='left', va='top', transform = ax3.transAxes)

E = ax4.text(0.01, 0.95, 'T5', fontsize=14, fontweight='bold', ha='left', va='top', transform = ax4.transAxes)

F = ax5.text(0.01, 0.95, 'T6', fontsize=14, fontweight='bold', ha='left', va='top', transform = ax5.transAxes)


#plt.savefig(r'results/final/bar_plot_bins_rolling_mean.png', bbox_inches='tight', format='png')


In [ ]:


df_sel_1 = df_sel_1[df_sel_1['bx_abs'] < 1.0E-6]
df_sel_2 = df_sel_2[df_sel_2['bx_abs'] < 1.0E-6]
df_sel_3 = df_sel_3[df_sel_3['bx_abs'] < 1.0E-6]
df_sel_4 = df_sel_4[df_sel_4['bx_abs'] < 1.0E-6]
df_sel_5 = df_sel_5[df_sel_5['bx_abs'] < 1.0E-6]
df_sel_6 = df_sel_6[df_sel_6['bx_abs'] < 1.0E-6]

In [ ]:
def plot_bars_ratio(df, df_base, ax, ax_twin, var2, var='lat', bin_size=0.1, rolling_value=4):
    df1 = df
    df2 = df_base
    bins_base = np.arange(round(df2[var].min(),2),round(df2[var].max(),2),bin_size)
    cats1, bins1 = pd.cut(x = df1['lat'], bins = bins_base, retbins=True)
    count1 = df1['bx_abs'].groupby(cats1).count()
    cats2, bins2 = pd.cut(x = df2['lat'], bins = bins_base, retbins=True)
    count2 = df2['bx_abs'].groupby(cats2).count()
    ratio = (count1/count2)*100
    rolling = ratio.rolling(rolling_value, center=True, closed='both').median()
    mdt = df2[var2].groupby(cats2).mean()
    #rolling2 = ratio.rolling(rolling_value, center=True, closed='both').median()
    
    #ax.bar(bins1[:-1], ratio, width=bin_size-0.02, bottom=0.01, align='center', color='black', ecolor='black', capsize=10)
    #ax.plot(bins1[:-1], ratio, color='blue')
    ax.boxplot(bins2)
    #ax_aux.plot(bins1[:-1], rolling2, c='blue', linewidth=4)
    #ax_twin.plot(bins1[:-1], mdt, c='darkorange', linewidth=4)

SAF_p = [SAF_1,SAF_2,SAF_3,SAF_4,SAF_5,SAF_6]
PF_p = [PF_1,PF_2,PF_3,PF_4,PF_5,PF_6]
SACCF_p = [SACCF_1,SACCF_2,SACCF_3,SACCF_4,SACCF_5,SACCF_6]

def front_lat(SAF, PF, SACCF, ax, color='red', marker='^', markersize=12, y_value=-0.005, zorder=500):
    ax.plot(SAF['lat'], y_value, marker=marker, markersize=markersize, c='red', zorder=zorder)
    ax.plot(PF['lat'], y_value, marker=marker, markersize=markersize, c='red', zorder=zorder)
    ax.plot(SACCF['lat'], y_value, marker=marker, markersize=markersize, c='red', zorder=zorder)

from matplotlib.gridspec import GridSpec
fig = plt.figure(figsize=(16,8), tight_layout=True)
gs = GridSpec(3,2)

ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1], sharex=ax0)
ax2 = fig.add_subplot(gs[1,0], sharex=ax0)
ax3 = fig.add_subplot(gs[1,1], sharex=ax0)
ax4 = fig.add_subplot(gs[2,0], sharex=ax0)
ax5 = fig.add_subplot(gs[2,1], sharex=ax0)
axes = [ax0, ax1, ax2, ax3, ax4, ax5]

ax_twin0 = ax0.twinx()
ax_twin1 = ax1.twinx()
ax_twin2 = ax2.twinx()
ax_twin3 = ax3.twinx()
ax_twin4 = ax4.twinx()
ax_twin5 = ax5.twinx()
twin_axes = [ax_twin0, ax_twin1, ax_twin2, ax_twin3, ax_twin4, ax_twin5]

#mdts = [mdt_1, mdt_2, mdt_3, mdt_4, mdt_5, mdt_6]

s = 0
dfs_filt = [df_sel_1_filt[s], df_sel_2_filt[s], df_sel_3_filt[s], df_sel_4_filt[s], df_sel_5_filt[s], df_sel_6_filt[s]]
#dfs_filt = [df_sel_1_filt, df_sel_2_filt, df_sel_3_filt, df_sel_4_filt, df_sel_5_filt, df_sel_6_filt]
dfs_base = [df_sel_1, df_sel_2, df_sel_3, df_sel_4, df_sel_5, df_sel_6]
for i in range(len(axes)):
    plot_bars_ratio(dfs_filt[i], dfs_base[i], axes[i], twin_axes[i], var2='adt_diff', bin_size=0.2, rolling_value=2)
    #front_lat(SAF_p[i], PF_p[i], SACCF_p[i], axes[i], y_value=0)
    
ax0.invert_xaxis()
ax4.set_xlabel('Latitude [°]', fontsize=12, fontweight='bold')
ax5.set_xlabel('Latitude [°]', fontsize=12, fontweight='bold')

ax0.set_ylabel('% of outliers per 0.1° bin', fontsize=10, fontweight='bold', c='dodgerblue')
ax2.set_ylabel('% of outliers per 0.1° bin', fontsize=10, fontweight='bold', c='dodgerblue')
ax4.set_ylabel('% of outliers per 0.1° bin', fontsize=10, fontweight='bold', c='dodgerblue')

ax_twin1.set_ylabel(r'$\mathrm{\frac{\partial \overline{SLA}}{\partial y}} \: [\frac{m}{°}]$', fontsize=20, fontweight='bold', c='darkorange')
ax_twin3.set_ylabel(r'$\mathrm{\frac{\partial \overline{SLA}}{\partial y}} \: [\frac{m}{°}]$', fontsize=20, fontweight='bold', c='darkorange')
ax_twin5.set_ylabel(r'$\mathrm{\frac{\partial \overline{SLA}}{\partial y}} \: [\frac{m}{°}]$', fontsize=20, fontweight='bold', c='darkorange')

A = ax0.text(0.01, 0.95, 'T1', fontsize=14, fontweight='bold', ha='left', va='top', transform = ax0.transAxes)

B = ax1.text(0.01, 0.95, 'T2', fontsize=14, fontweight='bold', ha='left', va='top', transform = ax1.transAxes)

C = ax2.text(0.01, 0.95, 'T3', fontsize=14, fontweight='bold', ha='left', va='top', transform = ax2.transAxes)

D = ax3.text(0.01, 0.95, 'T4', fontsize=14, fontweight='bold', ha='left', va='top', transform = ax3.transAxes)

E = ax4.text(0.01, 0.95, 'T5', fontsize=14, fontweight='bold', ha='left', va='top', transform = ax4.transAxes)

F = ax5.text(0.01, 0.95, 'T6', fontsize=14, fontweight='bold', ha='left', va='top', transform = ax5.transAxes)


#plt.savefig(r'results/final/bar_plot_bins_rolling_SLA.png', bbox_inches='tight', format='png')


In [ ]:
from matplotlib.gridspec import GridSpec
fig = plt.figure(constrained_layout=True, figsize=(16,8))
gs = GridSpec(2, 1, figure=fig) #, hspace=0.2,wspace=0.1)

df = df_all.sort_values('lat')[df_all['Tu'] > 0]

ax0 = fig.add_subplot(gs[0,0])
ax0.scatter(df['lat'], df['Tu'])
ax0.yaxis.set_major_locator(plt.MultipleLocator(np.pi / 2))
ax0.yaxis.set_minor_locator(plt.MultipleLocator(np.pi / 12))
ax0.set_yticks([-pi/2,-pi/4,0,pi/4,pi/2], labels=[r'$-\frac{\pi}{2}$', r'$-\frac{\pi}{4}$', 0, r'$\frac{\pi}{4}$', r'$\frac{\pi}{2}$'])
ax0.invert_xaxis()

ax1 = fig.add_subplot(gs[1,0])
ax1.plot(df['lat'], df['R'])
ax1.invert_xaxis()

In [ ]:
def dens_plot_2d(df, x_var, y_var, ax, nbins=100, cbar_loc='bottom'):
    from scipy.stats import kde
    x = df[x_var]
    y = df[y_var]
    nbins = nbins
    k = kde.gaussian_kde([x,y])
    xi, yi = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
    zi = k(np.vstack([xi.flatten(), yi.flatten()]))

    pc = ax.pcolormesh(xi, yi, zi.reshape(xi.shape), shading='auto', cmap='coolwarm')
    #fig.colorbar(pc, location=cbar_loc, pad=0.01, ax=ax)